In [24]:
import numpy as np
import cv2
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

class NeuralNet(object):

    def __init__(self):
        #загружаем датасет mnist
        (X_train,y_train), (X_test,y_test) = mnist.load_data()
        self.training_images = X_train.reshape((60000, 28 , 28,1)).astype('float32') / 255
        self.training_targets = to_categorical(y_train)
        self.test_images = X_test.reshape((10000, 28 , 28,1)).astype('float32') / 255
        self.test_targets = to_categorical(y_test)
        self.input_shape = (self.training_images.shape[1],)

        # создаем модельку
        self.model = Sequential()
        self.model.add(Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)))
        self.model.add(MaxPooling2D((2,2)))
        self.model.add(Conv2D(64, (3,3), activation='relu'))
        self.model.add(MaxPooling2D((2,2)))
        self.model.add(Conv2D(64, (3,3), activation='relu'))
        self.model.add(Flatten())
        self.model.add(Dense(64, activation='relu'))
        self.model.add(Dense(10, activation='softmax'))

        self.model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
        #тренируем(много раз запуская поняла, что для этого примера достаточно epochs)
        self.model.fit(self.training_images, self.training_targets, validation_split=0.3, epochs=5)

    def predict(self, image):
        #изначально тренировочные картинки имеют размеры 28x28, поэтому приводим и нарисованную картинку к этому виду
        input = cv2.resize(image, (28 , 28)).reshape((28 , 28,1)).astype('float32') / 255
        return self.model.predict_classes(np.array([input]))

In [25]:
net = NeuralNet()

Epoch 1/5
1313/1313 [==============================] - 36s 27ms/step - loss: 0.1710 - accuracy: 0.9466 - val_loss: 0.0808 - val_accuracy: 0.9759
Epoch 2/5
1313/1313 [==============================] - 32s 24ms/step - loss: 0.0514 - accuracy: 0.9838 - val_loss: 0.0583 - val_accuracy: 0.9822
Epoch 3/5
1313/1313 [==============================] - 31s 23ms/step - loss: 0.0361 - accuracy: 0.9886 - val_loss: 0.0502 - val_accuracy: 0.9858
Epoch 4/5
1313/1313 [==============================] - 37s 28ms/step - loss: 0.0297 - accuracy: 0.9907 - val_loss: 0.0396 - val_accuracy: 0.9878
Epoch 5/5
1313/1313 [==============================] - 39s 30ms/step - loss: 0.0238 - accuracy: 0.9921 - val_loss: 0.0397 - val_accuracy: 0.9884


In [34]:
import cv2
import numpy as np

canvas = np.ones((600,600), dtype="uint8") * 255
canvas[100:500,100:500] = 0

start_point = None
end_point = None
is_drawing = False

def draw_line(img,start_at,end_at):
    cv2.line(img,start_at,end_at,255,15)

def on_mouse_events(event,x,y,flags,params):
    global start_point
    global end_point
    global canvas
    global is_drawing
    if event == cv2.EVENT_LBUTTONDOWN:
        if is_drawing:
            start_point = (x,y)
    elif event == cv2.EVENT_MOUSEMOVE:
        if is_drawing:
            end_point = (x,y)
            draw_line(canvas,start_point,end_point)
            start_point = end_point
    elif event == cv2.EVENT_LBUTTONUP:
        is_drawing = False


cv2.namedWindow("Canvas")
cv2.setMouseCallback("Canvas", on_mouse_events)

#управление рисовалкой - на англ буквы:
#стирать нарисованное - c
#сообщать о готовности рисовать - s
#предсказать - p
#выйти (также сложно, как и в Vim редакторе))))) - q
#При первом запуске нажимаем: s, c и можно рисовать(а то канвас какую-то каляку в начале ставит, поэтому стираем)
while(True):
    cv2.imshow("Canvas", canvas)
    key = cv2.waitKey(1) & 0xFF 
    if key == ord('q'):
        break
    elif key == ord('s'):
        is_drawing = True
    elif key == ord('c'):
        canvas[100:500,100:500] = 0
    elif key == ord('p'):
        image = canvas[100:500,100:500]
        result = net.predict(image)
        print("PREDICTION : ",result)

cv2.destroyAllWindows()

PREDICTION :  [3]
PREDICTION :  [0]
PREDICTION :  [4]
PREDICTION :  [8]
PREDICTION :  [7]
PREDICTION :  [5]
